In [105]:
import pyprojroot

def make_dir_function(dir_name):
    def dir_function(*args):
        return pyprojroot.here().joinpath(dir_name, *args)

    return dir_function

In [106]:
data_dir = make_dir_function("Data")
data_dir("raw", "songs_normalize.csv")

WindowsPath('j:/DOCUEMENTOS/PROYECTOS PERSONALES/SPOTIFY/project/Data/raw/songs_normalize.csv')

In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics 
import seaborn as sns

In [108]:
df=pd.read_csv(data_dir("raw", "songs_normalize.csv"))

## ML model

In [109]:
data = df.copy()

In [110]:
data.head()

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop


In [111]:
delet_columns = ['song', 'energy', 'loudness', 'acousticness', 'instrumentalness', 'valence']
data = data.drop(delet_columns, axis=1)
data.head()

,artist,duration_ms,explicit,year,popularity,danceability,key,mode,speechiness,liveness,tempo,genre
0,Britney Spears,211160,False,2000,77,0.751,1,0,0.0437,0.3550,95.053,pop
1,blink-182,167066,False,1999,79,0.434,0,1,0.0488,0.6120,148.726,"rock, pop"
2,Faith Hill,250546,False,1999,66,0.529,7,1,0.0290,0.2510,136.859,"pop, country"
3,Bon Jovi,224493,False,2000,78,0.551,0,0,0.0466,0.3470,119.992,"rock, metal"
4,*NSYNC,200560,False,2000,65,0.614,8,0,0.0516,0.0845,172.656,pop


In [112]:
data_genre_freq = data['genre'].value_counts(normalize=True)
mapeo_genre = data_genre_freq.to_dict()
data['genre'] = data['genre'].map(mapeo_genre)

data_artist_freq = data['artist'].value_counts(normalize=True)
mapeo_artist = data_artist_freq.to_dict()
data['artist'] = data['artist'].map(mapeo_artist)

In [113]:
data = pd.get_dummies(data, columns=['explicit'], drop_first=True)

data.rename(columns={'explicit_True': 'explicit'}, inplace=True)

In [114]:
data.head()

,artist,duration_ms,year,popularity,danceability,key,mode,speechiness,liveness,tempo,genre,explicit
0,0.0095,211160,2000,77,0.751,1,0,0.0437,0.3550,95.053,0.2140,0
1,0.0005,167066,1999,79,0.434,0,1,0.0488,0.6120,148.726,0.0215,0
2,0.0010,250546,1999,66,0.529,7,1,0.0290,0.2510,136.859,0.0040,0
3,0.0005,224493,2000,78,0.551,0,0,0.0466,0.3470,119.992,0.0190,0
4,0.0020,200560,2000,65,0.614,8,0,0.0516,0.0845,172.656,0.2140,0


In [115]:
X = data.drop('explicit', axis=1)
y = data.danceability

X = X.values
y = y.values.reshape(-1,1)

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

sc_x = StandardScaler().fit(X)
X_train = sc_x.transform(X_train)
X_test = sc_x.transform(X_test)

sc_y = StandardScaler().fit(y)
y_train = sc_y.transform(y_train)
y_test = sc_y.transform(y_test)

In [117]:
model = LinearRegression()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

In [118]:
y_pred.shape

(600, 1)

## Evaluación del modelo

In [119]:
mse = metrics.mean_squared_error(y_test,y_pred)
r2 = metrics.r2_score(y_test, y_pred)

print('r2', r2.round(4))
print('mse: ', mse.round(4))

r2 1.0
mse:  0.0
